In [5]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "3" # choose which GPU you want to use

from tempfile import gettempdir
import urllib
import zipfile
import numpy as np
import random
import matplotlib.pyplot as plt

import tensorflow as tf
print(tf.__version__)

1.11.0


In [6]:
from tensorflow.examples.tutorials.mnist import input_data
# load mnist data
mnist = input_data.read_data_sets("data", one_hot=True)


Extracting data\train-images-idx3-ubyte.gz
Extracting data\train-labels-idx1-ubyte.gz
Extracting data\t10k-images-idx3-ubyte.gz
Extracting data\t10k-labels-idx1-ubyte.gz


In [162]:
# setting 
feature_dims = 2
neurons = 5
classes = 1
learning_rate = 0.1
def fully_connected_layer(x_inputs, out_dim, name='fc'):
    """ Low level method
        x_inputs: a batch examples [batch_size, feature_dims]
        out_dim: neurons in this layer.
    """ 
    in_dim = x_inputs.shape[-1] # feature_dims
    with tf.variable_scope(name, reuse=tf.AUTO_REUSE):
        weights = tf.get_variable("weights", shape=[in_dim, out_dim])
        bias = tf.get_variable("bias", shape=[out_dim])
        out = tf.matmul(x_inputs, weights) + bias
        return out
# xor task
xor_data = np.array([[1, 0],
                    [0, 1],
                    [1, 1],
                    [0, 0]])
xor_label = np.array([[1], [1], [0], [0]])

with tf.Graph().as_default() as g:
    x_input = tf.placeholder(tf.float32, [None, 2])
    y_label = tf.placeholder(tf.float32, [None, 1])
    # start building your model and meet the requirements
    # from here
    
    out = fully_connected_layer(x_input,6,'L1')
    out = fully_connected_layer( tf.sigmoid(out),1, "L2")
    out = tf.sigmoid(out)

    loss = tf.reduce_mean((out-y_label)**2, name="Logist")
    optimizer = tf.train.AdamOptimizer(learning_rate)
    
    training_op = optimizer.minimize(loss,name = "training_op")
    init = tf.global_variables_initializer()
    

with tf.Session(graph=g) as sess:
    # start run the seesion and meet the requrements
    # from here
    sess.run(init)
    for i in range(100):
        sess.run([loss,training_op],feed_dict={x_input: xor_data,y_label: xor_label})
        
    x = sess.run(out,feed_dict={x_input: xor_data})
    with tf.variable_scope("L1", reuse=True):
        print('full connected layer 1 w')
        v = tf.get_variable("weights")
        print(sess.run(v))
        
        print('full connected layer 1 bias')
        v = tf.get_variable("bias")
        print(sess.run(v))
    with tf.variable_scope("L2", reuse=True):
        print('full connected layer 1 w')
        v = tf.get_variable("weights")
        print(sess.run(v))
        
        print('full connected layer 1 bias')
        v = tf.get_variable("bias")
        print(sess.run(v))
        
#         vs.
#     var = [v for v in tf.trainable_variables()][0]
    print('acc:',sum((xor_label==np.round(x))/4)[0])
    

full connected layer 1 w
[[-6.2220893  -0.53706723  2.9599993  -5.7578382   6.3434434  -6.9728923 ]
 [ 4.6419377  -0.22326803 -5.282519    7.6141953  -4.604753   -6.415772  ]]
full connected layer 1 bias
[-1.9162577  -0.90517175 -0.7304128   2.4763248   1.9582983   2.0814915 ]
full connected layer 1 w
[[ 4.6504145 ]
 [ 0.54976463]
 [ 3.1749053 ]
 [-3.4743433 ]
 [-1.9929037 ]
 [-3.0789042 ]]
full connected layer 1 bias
[2.1787846]
acc: 1.0


[Problem Definition] <br>
Build a fully connected network to challenge this task.<br>
Input: an image of shape=[784]<br>
Output: a digit of this image, shape=[10]

[Requirements]<br>
Show the code of graph and session you build.<br>
Use tf.data.Dataset and tf.data.Iterator to extract data.<br>
Use low level API method to build the model (define the weights and bias from scratch like above).<br>
The accuracy on mnist.test should be at least 95%.

[Notes]<br>
the hyperparameters are not constrainted (e.g. num_neurons_in_one_layer, how_many_layers, learning_rates, training_epochs, batch_size)<br>
the optimizer are not constrainted.<br>
mnist.train is all you can use to train the model. mnist.validation are just used to be validated.

In [253]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.contrib.framework import arg_scope
from tensorflow.contrib.layers import fully_connected, batch_norm
from tensorflow.examples.tutorials.mnist import input_data

###### Do not modify here ###### 
def fully_connected_layer(x_inputs, out_dim, name='fc'):
    """ Low level method
        x_inputs: a batch examples [batch_size, feature_dims]
        out_dim: neurons in this layer.
    """ 
    in_dim = x_inputs.shape[-1] # feature_dims
    
    with tf.variable_scope(name, reuse=tf.AUTO_REUSE):
        weights = tf.get_variable("weights", shape=[in_dim, out_dim])
        bias = tf.get_variable("bias", shape=[out_dim])
        out = tf.matmul(x_inputs, weights) + bias
        return out
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()


# mnist = input_data.read_data_sets("data", one_hot=True)
# mnist = input_data.read_data_sets("data")
if not mnist:
    mnist = input_data.read_data_sets("data")


# training on MNIST but only on digits 0 to 4
X_train1 = mnist.train.images
y_train1 = mnist.train.labels

X_test1 = mnist.test.images
y_test1 = mnist.test.labels

###### Do not modify here ###### 

#get next batch in order
def next_batch(batch_size,iteration, data, labels):
    start = batch_size*iteration
    end = batch_size*(iteration+1)
    return data[start:end], labels[start:end]

#const parameters
n_inputs = 784  # MNIST
n_outputs = 10

#adjustable parameters
N_neurons = 128
learning_rate = 0.01
momentum = 0.25
epochs = 2
batch_size = 128  #for GPU optimize
dropout = 0.5


with tf.Graph().as_default() as g:
    training_dataset = tf.data.Dataset.from_tensor_slices((mnist.train.images,mnist.train.labels)) 
    training_dataset = training_dataset.batch(batch_size)
    train_iterator = training_dataset.make_one_shot_iterator()


    test_dataset = tf.data.Dataset.from_tensor_slices((mnist.test.images,mnist.test.labels))
    test_dataset = test_dataset.batch(batch_size)
    test_iterator = test_dataset.make_one_shot_iterator()
    is_test = tf.placeholder(tf.bool, shape=())

    X,y = tf.cond(is_test, lambda:test_iterator.get_next(), lambda:train_iterator.get_next())
    print(X)
    # X = tf.manip.reshape(X,(-1,784))
    # # X = tf.transpose(X)
    y= tf.cast(y, tf.int64)
    # y = tf.manip.reshape(y,(-1,))

    # X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
    # y = tf.placeholder(tf.int64, shape=(None), name="y")
    # print(X) #(?,784)

    he_init = tf.contrib.layers.variance_scaling_initializer()

    W1 = tf.sigmoid(fully_connected_layer(X,N_neurons,name='L1')) 
    # print(W1) #(?,128)

    W2 = tf.sigmoid(fully_connected_layer(W1,N_neurons,name='L2'))
    # print(W2) #(?,128)

    y_hat = fully_connected_layer(W2, n_outputs,name = 'L3')
    # print(y_hat) #(?,10)
    
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=y_hat)
    loss = tf.reduce_mean(cross_entropy, name="loss")

    #use AdamOptimizer
    optimizer = tf.train.AdamOptimizer(learning_rate, momentum)
    training_op = optimizer.minimize(loss,name = "training_op")


    correct = tf.nn.in_top_k(y_hat,y,1)
    accuracy = tf.reduce_mean(tf.cast(correct,tf.float32), name="accuracy")


    init = tf.global_variables_initializer()

with tf.Session(graph=g) as sess:
    init.run()
    
    for epoch in range(epochs):
        for iteration in range(50):
            sess.run(training_op,feed_dict={is_test:False} )            

    acc_test = sess.run(accuracy,feed_dict={is_test:True} )
        
    print( "Test accuracy:", acc_test)

Tensor("cond/Merge:0", shape=(?, 784), dtype=float32)
Test accuracy: 0.953125
